In [11]:
import sqlite3

conn = sqlite3.connect(r'D:\2025\Labmentix\05_Vaccination report\vaccination-project\db\vaccination.db')
cursor = conn.cursor()

# List tables
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
print("Tables:", cursor.fetchall())

# Check columns for each table
tables = ['coverage', 'country', 'vaccine', 'incidence', 'reported_cases', 'disease', 'vaccine_introduction']
for table in tables:
    cursor.execute(f"PRAGMA table_info({table});")
    print(f"\nColumns in {table}:")
    for col in cursor.fetchall():
        print(col)

conn.close()


Tables: [('country',), ('sqlite_sequence',), ('vaccine',), ('disease',), ('coverage',), ('incidence',), ('reported_cases',), ('vaccine_introduction',), ('vaccine_schedule',)]

Columns in coverage:
(0, 'id', 'INTEGER', 0, None, 1)
(1, 'country_id', 'INTEGER', 0, None, 0)
(2, 'vaccine_id', 'INTEGER', 0, None, 0)
(3, 'year', 'INTEGER', 0, None, 0)
(4, 'coverage_category', 'TEXT', 0, None, 0)
(5, 'coverage_category_description', 'TEXT', 0, None, 0)
(6, 'target_number', 'INTEGER', 0, None, 0)
(7, 'doses_administered', 'INTEGER', 0, None, 0)
(8, 'coverage', 'REAL', 0, None, 0)
(9, 'coverage_percent', 'REAL', 0, None, 0)

Columns in country:
(0, 'id', 'INTEGER', 0, None, 1)
(1, 'iso3', 'CHAR(3)', 0, None, 0)
(2, 'name', 'TEXT', 0, None, 0)
(3, 'who_region', 'TEXT', 0, None, 0)

Columns in vaccine:
(0, 'id', 'INTEGER', 0, None, 1)
(1, 'code', 'TEXT', 0, None, 0)
(2, 'description', 'TEXT', 0, None, 0)

Columns in incidence:
(0, 'id', 'INTEGER', 0, None, 1)
(1, 'country_id', 'INTEGER', 0, None, 

In [16]:
import sqlite3
import pandas as pd

# ------------------------------
# Connect to the SQLite database
# ------------------------------
db_path = r'D:/2025/Labmentix/05_Vaccination report/vaccination-project/db/vaccination.db'
conn = sqlite3.connect(db_path)
cursor = conn.cursor()

# ------------------------------
# Detect the correct incidence column
# ------------------------------
cursor.execute("PRAGMA table_info(incidence);")
inc_columns = cursor.fetchall()
incidence_col = None
for col in inc_columns:
    if 'incidence' in col[1].lower():
        incidence_col = col[1]
        break
if incidence_col is None:
    raise ValueError("No incidence column found in the incidence table!")

# ------------------------------
# Use the actual reported_cases column
# ------------------------------
reported_cases_col = "cases"  # fixed column name

# ------------------------------
# Define all 25 queries dynamically
# ------------------------------
queries = {
    1: "SELECT c.year, co.name AS country_name, v.description AS vaccine_name, c.coverage_percent "
       "FROM coverage c "
       "JOIN country co ON c.country_id = co.id "
       "JOIN vaccine v ON c.vaccine_id = v.id "
       "ORDER BY c.year;",

    2: "SELECT co.name AS country_name, AVG(c.coverage_percent) AS avg_coverage "
       "FROM coverage c "
       "JOIN country co ON c.country_id = co.id "
       "GROUP BY co.name "
       "ORDER BY avg_coverage DESC;",

    3: "SELECT v.description AS vaccine_name, AVG(c.coverage_percent) AS avg_coverage "
       "FROM coverage c "
       "JOIN vaccine v ON c.vaccine_id = v.id "
       "GROUP BY v.description "
       "ORDER BY avg_coverage DESC;",

    4: "SELECT c.year, v.description AS vaccine_name, c.coverage_percent "
       "FROM coverage c "
       "JOIN country co ON c.country_id = co.id "
       "JOIN vaccine v ON c.vaccine_id = v.id "
       "WHERE co.name = 'India' "
       "ORDER BY c.year;",

    5: "SELECT v.description AS vaccine_name, co.name AS country_name, AVG(c.coverage_percent) AS avg_coverage "
       "FROM coverage c "
       "JOIN country co ON c.country_id = co.id "
       "JOIN vaccine v ON c.vaccine_id = v.id "
       "GROUP BY v.description, co.name "
       "ORDER BY v.description, avg_coverage DESC "
       "LIMIT 5;",

    6: f"SELECT d.description AS disease_name, i.year, SUM(i.{incidence_col}) AS total_incidence "
       f"FROM incidence i "
       f"JOIN disease d ON i.disease_id = d.id "
       f"GROUP BY d.description, i.year "
       f"ORDER BY d.description, i.year;",

    7: f"SELECT d.description AS disease_name, i.year, i.{incidence_col} "
       f"FROM incidence i "
       f"JOIN disease d ON i.disease_id = d.id "
       f"JOIN country co ON i.country_id = co.id "
       f"WHERE co.name = 'India' "
       f"ORDER BY i.year;",

    8: f"SELECT co.name AS country_name, SUM(i.{incidence_col}) AS total_incidence "
       f"FROM incidence i "
       f"JOIN country co ON i.country_id = co.id "
       f"GROUP BY co.name "
       f"ORDER BY total_incidence DESC;",

    9: f"SELECT co.name AS country_name, SUM(rc.{reported_cases_col}) AS total_cases "
       f"FROM reported_cases rc "
       f"JOIN country co ON rc.country_id = co.id "
       f"GROUP BY co.name "
       f"ORDER BY total_cases DESC;",

    10: f"SELECT d.description AS disease_name, SUM(rc.{reported_cases_col}) AS total_cases "
        f"FROM reported_cases rc "
        f"JOIN disease d ON rc.disease_id = d.id "
        f"GROUP BY d.description "
        f"ORDER BY total_cases DESC;",

    11: f"SELECT co.name AS country_name, d.description AS disease_name, SUM(rc.{reported_cases_col}) AS total_cases "
        f"FROM reported_cases rc "
        f"JOIN country co ON rc.country_id = co.id "
        f"JOIN disease d ON rc.disease_id = d.id "
        f"GROUP BY co.name, d.description "
        f"ORDER BY co.name, total_cases DESC;",

    12: f"SELECT co.name AS country_name, v.description AS vaccine_name, c.coverage_percent, i.{incidence_col} "
        f"FROM coverage c "
        f"JOIN vaccine v ON c.vaccine_id = v.id "
        f"JOIN country co ON c.country_id = co.id "
        f"LEFT JOIN incidence i ON co.id = i.country_id "
        f"WHERE co.name = 'India';",

    13: f"SELECT co.name AS country_name, AVG(c.coverage_percent) AS avg_coverage, SUM(i.{incidence_col}) AS total_incidence "
        f"FROM coverage c "
        f"JOIN country co ON c.country_id = co.id "
        f"LEFT JOIN incidence i ON co.id = i.country_id "
        f"GROUP BY co.name "
        f"ORDER BY avg_coverage DESC;",

    14: "SELECT * FROM coverage WHERE country_id IS NULL OR vaccine_id IS NULL;",
    15: "SELECT * FROM incidence WHERE country_id IS NULL OR disease_id IS NULL;",

    16: "SELECT v.description AS vaccine_name, AVG(c.coverage_percent) AS avg_coverage "
        "FROM coverage c "
        "JOIN vaccine v ON c.vaccine_id = v.id "
        "GROUP BY v.description "
        "ORDER BY avg_coverage ASC;",

    17: f"SELECT d.description AS disease_name, SUM(i.{incidence_col}) AS total_incidence "
        f"FROM incidence i "
        f"JOIN disease d ON i.disease_id = d.id "
        f"GROUP BY d.description "
        f"ORDER BY total_incidence DESC;",

    18: f"SELECT co.name AS country_name, SUM(rc.{reported_cases_col}) AS total_cases "
        f"FROM reported_cases rc "
        f"JOIN country co ON rc.country_id = co.id "
        f"JOIN disease d ON rc.disease_id = d.id "
        f"WHERE d.description = 'Measles' "
        f"GROUP BY co.name "
        f"ORDER BY total_cases DESC;",

    19: "SELECT c.year, AVG(c.coverage_percent) AS avg_coverage FROM coverage c "
        "GROUP BY c.year "
        "ORDER BY c.year;",

    20: f"SELECT i.year, SUM(i.{incidence_col}) AS total_incidence FROM incidence i "
        "GROUP BY i.year "
        "ORDER BY i.year;",

    21: f"SELECT co.name AS country_name, v.description AS vaccine_name, AVG(c.coverage_percent) AS avg_coverage, "
        f"SUM(i.{incidence_col}) AS total_incidence, "
        f"(AVG(c.coverage_percent) / NULLIF(SUM(i.{incidence_col}),0)) AS coverage_to_incidence_ratio "
        "FROM coverage c "
        "JOIN country co ON c.country_id = co.id "
        "JOIN vaccine v ON c.vaccine_id = v.id "
        f"LEFT JOIN incidence i ON co.id = i.country_id "
        "GROUP BY co.name, v.description "
        "ORDER BY coverage_to_incidence_ratio DESC;",

    22: "SELECT co.name AS country_name, v.description AS vaccine_name, c.year, AVG(c.coverage_percent) AS avg_coverage, "
        "(AVG(c.coverage_percent) - LAG(AVG(c.coverage_percent)) OVER (PARTITION BY co.name, v.description ORDER BY c.year)) AS coverage_growth "
        "FROM coverage c "
        "JOIN country co ON c.country_id = co.id "
        "JOIN vaccine v ON c.vaccine_id = v.id "
        "GROUP BY co.name, v.description, c.year "
        "ORDER BY co.name, v.description, c.year;",

    23: f"SELECT co.name AS country_name, v.description AS vaccine_name, AVG(c.coverage_percent) AS avg_coverage, "
        f"SUM(rc.{reported_cases_col}) AS total_cases "
        "FROM coverage c "
        "JOIN country co ON c.country_id = co.id "
        "JOIN vaccine v ON c.vaccine_id = v.id "
        f"LEFT JOIN reported_cases rc ON co.id = rc.country_id "
        "GROUP BY co.name, v.description "
        "ORDER BY avg_coverage DESC;",

    24: "SELECT co.name AS country_name, v.description AS vaccine_name, COUNT(*) AS missing_data_count "
        "FROM coverage c "
        "JOIN country co ON c.country_id = co.id "
        "JOIN vaccine v ON c.vaccine_id = v.id "
        "WHERE c.coverage_percent IS NULL "
        "GROUP BY co.name, v.description "
        "ORDER BY missing_data_count DESC;",

    25: "SELECT co.name AS country_name, v.description AS vaccine_name, MIN(vi.year) AS first_introduction, "
        "MAX(vi.year) AS last_introduction "
        "FROM vaccine_introduction vi "
        "JOIN country co ON vi.country_id = co.id "
        "JOIN vaccine v ON vi.vaccine_id = v.id "
        "GROUP BY co.name, v.description "
        "ORDER BY first_introduction;"
}

# ------------------------------
# Execute all queries and save CSVs
# ------------------------------
for idx, q in queries.items():
    try:
        cursor.execute(q)
        rows = cursor.fetchall()
        df = pd.DataFrame(rows, columns=[desc[0] for desc in cursor.description])
        df.to_csv(f'D:/2025/Labmentix/05_Vaccination report/vaccination-project/data/query_{idx}.csv', index=False)
        print(f"Query {idx} executed successfully!")
    except Exception as e:
        print(f"Query {idx} failed: {e}")

# ------------------------------
# Close connection
# ------------------------------
conn.close()
print("All 25 queries executed!")


Query 1 executed successfully!
Query 2 executed successfully!
Query 3 executed successfully!
Query 4 executed successfully!
Query 5 executed successfully!
Query 6 executed successfully!
Query 7 executed successfully!
Query 8 executed successfully!
Query 9 executed successfully!
Query 10 executed successfully!
Query 11 executed successfully!
Query 12 executed successfully!
Query 13 executed successfully!
Query 14 executed successfully!
Query 15 executed successfully!
Query 16 executed successfully!
Query 17 executed successfully!
Query 18 executed successfully!
Query 19 executed successfully!
Query 20 executed successfully!
Query 21 executed successfully!
Query 22 executed successfully!
Query 23 executed successfully!
Query 24 executed successfully!
Query 25 executed successfully!
All 25 queries executed!


In [20]:
# ============================================================
# Run SQL Queries Notebook - Fully Error-Proof
# ============================================================

import sqlite3
import pandas as pd
import os

# -------------------------------
# Step 1: Set paths
# -------------------------------
db_path = r'D:/2025/Labmentix/05_Vaccination report/vaccination-project/db/vaccination.db'
sql_file = r'D:/2025/Labmentix/05_Vaccination report/vaccination-project/sql/05_core_sql_queries.sql'
data_folder = r'D:/2025/Labmentix/05_Vaccination report/vaccination-project/data'

# Create data folder if it doesn't exist
os.makedirs(data_folder, exist_ok=True)

# -------------------------------
# Step 2: Connect to the SQLite database
# -------------------------------
conn = sqlite3.connect(db_path)
cursor = conn.cursor()

# -------------------------------
# Step 3: Detect incidence column in incidence table
# -------------------------------
cursor.execute("PRAGMA table_info(incidence);")
inc_columns = cursor.fetchall()
incidence_col = None
for col in inc_columns:
    if 'incidence' in col[1].lower():
        incidence_col = col[1]
        break
if incidence_col is None:
    raise ValueError("No incidence column found in the incidence table!")

# -------------------------------
# Step 4: Detect reported_cases column
# -------------------------------
cursor.execute("PRAGMA table_info(reported_cases);")
rc_columns = cursor.fetchall()
reported_cases_col = None
for col in rc_columns:
    if 'case' in col[1].lower():
        reported_cases_col = col[1]
        break
if reported_cases_col is None:
    raise ValueError("No reported_cases column found in the reported_cases table!")

# -------------------------------
# Step 5: Read SQL file and replace placeholders
# -------------------------------
with open(sql_file, 'r') as f:
    sql_content = f.read()

sql_content = sql_content.replace("{incidence_col}", incidence_col)
sql_content = sql_content.replace("{reported_cases_col}", reported_cases_col)

# Split queries by semicolon
queries = [q.strip() for q in sql_content.split(';') if q.strip()]

# -------------------------------
# Step 6: Execute all queries and save CSVs
# -------------------------------
for i, query in enumerate(queries):
    try:
        df = pd.read_sql_query(query, conn)
        csv_path = os.path.join(data_folder, f'query_{i+1}.csv')
        df.to_csv(csv_path, index=False)
        print(f'Query {i+1} executed successfully, saved to {csv_path}')
    except Exception as e:
        print(f'Query {i+1} failed: {e}')

# -------------------------------
# Step 7: Close connection
# -------------------------------
conn.close()
print("All queries processed!")


Query 1 executed successfully, saved to D:/2025/Labmentix/05_Vaccination report/vaccination-project/data\query_1.csv
Query 2 executed successfully, saved to D:/2025/Labmentix/05_Vaccination report/vaccination-project/data\query_2.csv
Query 3 executed successfully, saved to D:/2025/Labmentix/05_Vaccination report/vaccination-project/data\query_3.csv
Query 4 executed successfully, saved to D:/2025/Labmentix/05_Vaccination report/vaccination-project/data\query_4.csv
Query 5 executed successfully, saved to D:/2025/Labmentix/05_Vaccination report/vaccination-project/data\query_5.csv
Query 6 executed successfully, saved to D:/2025/Labmentix/05_Vaccination report/vaccination-project/data\query_6.csv
Query 7 executed successfully, saved to D:/2025/Labmentix/05_Vaccination report/vaccination-project/data\query_7.csv
Query 8 executed successfully, saved to D:/2025/Labmentix/05_Vaccination report/vaccination-project/data\query_8.csv
Query 9 executed successfully, saved to D:/2025/Labmentix/05_Vac

In [24]:
import sqlite3
import pandas as pd
import os

# -------------------------------
# Step 1: Set paths
# -------------------------------
db_path = r'D:/2025/Labmentix/05_Vaccination report/vaccination-project/db/vaccination.db'
data_folder = r'D:/2025/Labmentix/05_Vaccination report/vaccination-project/data'
os.makedirs(data_folder, exist_ok=True)

# -------------------------------
# Step 2: Connect to SQLite database
# -------------------------------
conn = sqlite3.connect(db_path)
cursor = conn.cursor()

# -------------------------------
# Step 3: Detect correct columns
# -------------------------------
# For incidence table
cursor.execute("PRAGMA table_info(incidence);")
inc_columns = cursor.fetchall()
incidence_col = None
for col in inc_columns:
    if 'incidence' in col[1].lower():
        incidence_col = col[1]
        break
if incidence_col is None:
    raise ValueError("No incidence column found in the incidence table!")

# For reported_cases table
cursor.execute("PRAGMA table_info(reported_cases);")
rc_columns = cursor.fetchall()
reported_cases_col = None
for col in rc_columns:
    if 'case' in col[1].lower():  # detect columns like 'cases', 'reported_cases'
        reported_cases_col = col[1]
        break
if reported_cases_col is None:
    raise ValueError("No reported_cases column found in the reported_cases table!")

print(f"Using incidence column: {incidence_col}")
print(f"Using reported_cases column: {reported_cases_col}")

# -------------------------------
# Step 4: Define all 25 queries dynamically
# Replace placeholders {incidence_col} and {reported_cases_col} dynamically
# -------------------------------
queries = {
    1: "SELECT c.year, co.name AS country_name, v.description AS vaccine_name, c.coverage_percent "
       "FROM coverage c "
       "JOIN country co ON c.country_id = co.id "
       "JOIN vaccine v ON c.vaccine_id = v.id "
       "ORDER BY c.year;",

    2: "SELECT co.name AS country_name, AVG(c.coverage_percent) AS avg_coverage "
       "FROM coverage c "
       "JOIN country co ON c.country_id = co.id "
       "GROUP BY co.name "
       "ORDER BY avg_coverage DESC;",

    6: f"SELECT d.description AS disease_name, i.year, SUM(i.{incidence_col}) AS total_incidence "
       f"FROM incidence i "
       f"JOIN disease d ON i.disease_id = d.id "
       f"GROUP BY d.description, i.year "
       f"ORDER BY d.description, i.year;",

    7: f"SELECT d.description AS disease_name, i.year, i.{incidence_col} "
       f"FROM incidence i "
       f"JOIN disease d ON i.disease_id = d.id "
       f"JOIN country co ON i.country_id = co.id "
       f"WHERE co.name = 'India' "
       f"ORDER BY i.year;",

    9: f"SELECT co.name AS country_name, SUM(rc.{reported_cases_col}) AS total_cases "
       f"FROM reported_cases rc "
       f"JOIN country co ON rc.country_id = co.id "
       f"GROUP BY co.name "
       f"ORDER BY total_cases DESC;",

    10: f"SELECT d.description AS disease_name, SUM(rc.{reported_cases_col}) AS total_cases "
        f"FROM reported_cases rc "
        f"JOIN disease d ON rc.disease_id = d.id "
        f"GROUP BY d.description "
        f"ORDER BY total_cases DESC;"
}

# Add remaining queries (3-5, 8, 11-25) similarly with {incidence_col} and {reported_cases_col} placeholders

# -------------------------------
# Step 5: Run queries with automatic fallback
# -------------------------------
summary = []

for idx, q in queries.items():
    try:
        df = pd.read_sql_query(q, conn)
        csv_path = os.path.join(data_folder, f'query_{idx}.csv')
        df.to_csv(csv_path, index=False)
        status = "Success"
    except Exception as e:
        # Automatic fallback: if query fails due to missing column, try alternatives
        if 'no such column' in str(e).lower():
            # Replace with detected correct columns dynamically
            q_fixed = q.replace('i.incidence', f'i.{incidence_col}').replace('rc.reported_cases', f'rc.{reported_cases_col}')
            try:
                df = pd.read_sql_query(q_fixed, conn)
                csv_path = os.path.join(data_folder, f'query_{idx}.csv')
                df.to_csv(csv_path, index=False)
                status = f"Success (fixed column)"
            except Exception as e2:
                csv_path = ""
                status = f"Failed after fix: {e2}"
        else:
            csv_path = ""
            status = f"Failed: {e}"

    summary.append({
        "Query Number": idx,
        "Status": status,
        "CSV Path": csv_path
    })
    print(f"Query {idx}: {status}")

# -------------------------------
# Step 6: Summary CSV
# -------------------------------
summary_df = pd.DataFrame(summary).sort_values("Query Number")
summary_df.to_csv(os.path.join(data_folder, "query_summary.csv"), index=False)
display(summary_df)

# -------------------------------
# Step 7: Close connection
# -------------------------------
conn.close()
print("All done!")


Using incidence column: incidence_rate
Using reported_cases column: cases
Query 1: Success
Query 2: Success
Query 6: Success
Query 7: Success
Query 9: Success
Query 10: Success


,Query Number,Status,CSV Path
0,1,Success,D:/2025/Labmentix/05_Vaccination report/vaccin...
1,2,Success,D:/2025/Labmentix/05_Vaccination report/vaccin...
2,6,Success,D:/2025/Labmentix/05_Vaccination report/vaccin...
3,7,Success,D:/2025/Labmentix/05_Vaccination report/vaccin...
4,9,Success,D:/2025/Labmentix/05_Vaccination report/vaccin...
5,10,Success,D:/2025/Labmentix/05_Vaccination report/vaccin...


All done!
